In [ ]:
!pip install ktrain

In [2]:
!wget https://raw.githubusercontent.com/mayanksinghkgp/NLP_Resources/main/FastText_MultiLabel/data/train.csv

--2021-12-22 11:00:29--  https://raw.githubusercontent.com/mayanksinghkgp/NLP_Resources/main/FastText_MultiLabel/data/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68802655 (66M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  65.62M  93.6MB/s    in 0.7s    

2021-12-22 11:00:35 (93.6 MB/s) - ‘train.csv’ saved [68802655/68802655]



In [3]:
import pandas as pd
import ktrain
from ktrain import text

In [4]:
PATH = 'train.csv'
NUM_WORDS = 50000     #Number of words in the vocal including ngrams etc
MAXLEN = 150      #Max length of the comment considered

train, val, preproc = text.texts_from_csv(PATH, 'comment_text', label_columns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'],
                    ngram_range = 1, max_features = NUM_WORDS, maxlen = MAXLEN)

detected encoding: utf-8 (if wrong, set manually)
['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
        toxic  severe_toxic  obscene  threat  insult  identity_hate
100541      0             0        0       0       0              0
129782      0             0        0       0       0              0
54248       1             0        0       0       0              0
80205       0             0        0       0       0              0
86120       0             0        0       0       0              0
['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
        toxic  severe_toxic  obscene  threat  insult  identity_hate
118031      0             0        0       0       0              0
83216       0             0        0       0       0              0
122548      0             0        0       0       0              0
157518      0             0        0       0       0              0
9171        0             0        0       0       0  

In [5]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) from keras_bert [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face transformers [https://arxiv.org/abs/1910.01108]


In [6]:
model = text.text_classifier('fasttext', train, preproc)

Is Multi-Label? True
compiling word ID features...
maxlen is 150
done.


In [7]:
learner = ktrain.get_learner(model, train, val)

In [ ]:
learner.autofit(0.001, 10)



begin training using triangular learning rate policy with max lr of 0.001...
Epoch 1/10
3034/4488 [===================>..........] - ETA: 15s - loss: 0.0584 - accuracy: 0.9941

In [10]:
predictor  = ktrain.get_predictor(learner.model, preproc)

In [13]:
predictor.predict(['I will make your entire family cry'])

[[('toxic', 0.26357433),
  ('severe_toxic', 0.0013144354),
  ('obscene', 0.04567338),
  ('threat', 0.0106521435),
  ('insult', 0.06905146),
  ('identity_hate', 0.0134433415)]]